## Лабораторная №2

In [1]:
DIRECTORY="./src"

cd "$DIRECTORY" || exit

rm -rf *.out

gcc task1.c wraper.c -o task1.out -Wall -Wextra

gcc task2.c wraper.c -o task2.out -Wall -Wextra

gcc task34567.c wraper.c -o task3.out -Wall -Wextra
ln task3.out task4.out
ln task3.out task5.out
ln task3.out task6.out
ln task3.out task7.out

gcc task89.c wraper.c -o task8.out -Wall -Wextra
ln task8.out task9.out

gcc task10.c wraper.c -o task10.out -Wall -Wextra

gcc task11.c wraper.c -o task11.out -Wall -Wextra

# Задание 1

Написать программу, которая добавляет новые строки в среду процесса.

```c
// task1.c
#include <stdio.h>
#include <stdlib.h>
#include <errno.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <fcntl.h>

#include "wraper.h"

extern char ** environ;

const char *const red = "\033[31m";
const char *const reset = "\033[0m";

int main(int argc, char * argv[], char * envp[]) {
    int i;

    if (argc != 1) {
        _fprintf(stderr, "Usage: %s\n",
                 argv[0]);
        exit(1);
    }

    _printf("%sBefore add%s\n", red, reset);
    for (i = 0; envp[i]; i++)
        _printf("%s\n", envp[i]);

    _putenv("A=AAA");
    _putenv("B=BBB");
    
    _printf("%sAfter add%s\n", red, reset);
    for (i = 0; envp[i]; i++)
        _printf("%s\n", envp[i]);
    
    _printf("%sAfter add with environ%s\n", red, reset);
    for (i = 0; environ[i]; i++)
        _printf("%s\n", environ[i]);
    
    exit(0);
}
```

In [2]:
./task1.out

Before add
SHELL=/bin/bash
SESSION_MANAGER=local/MSI-Desktop:@/tmp/.ICE-unix/9436,unix/MSI-Desktop:/tmp/.ICE-unix/9436
QT_ACCESSIBILITY=1
SNAP_REVISION=206
XDG_CONFIG_DIRS=/etc/xdg/xdg-ubuntu:/etc/xdg
PYTHONUNBUFFERED=1
XDG_MENU_PREFIX=gnome-
XDG_CONFIG_DIRS_VSCODE_SNAP_ORIG=/etc/xdg/xdg-ubuntu:/etc/xdg
GNOME_DESKTOP_SESSION_ID=this-is-deprecated
SNAP_REAL_HOME=/home/alexander
SNAP_USER_COMMON=/home/alexander/snap/code/common
FONTCONFIG_PATH=/etc/fonts
GNOME_SHELL_SESSION_MODE=ubuntu
SSH_AUTH_SOCK=/run/user/1000/keyring/ssh
XDG_DATA_HOME=/home/alexander/snap/code/206/.local/share
MEMORY_PRESSURE_WRITE=c29tZSAyMDAwMDAgMjAwMDAwMAA=
PYTHON_FROZEN_MODULES=on
ELECTRON_RUN_AS_NODE=1
XMODIFIERS=@im=ibus
DESKTOP_SESSION=ubuntu
GDK_PIXBUF_MODULE_FILE=/home/alexander/snap/code/common/.cache/gdk-pixbuf-loaders.cache
BAMF_DESKTOP_FILE_HINT=/var/lib/snapd/desktop/applications/code_code.desktop
GTK_MODULES=gail:atk-bridge
SNAP_EUID=1000
PWD=/home/alexander/Documents/AOS/lab2/src
GSETTINGS_SCHEMA_DIR

# Задание 2

Кто выполняется первым после fork: отец или сын?

Написать программу, которую можно использовать для проверки данной ситуации.

```c
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>

#include "wraper.h"

int main() {
    int ret_val;
    if ((ret_val = _fork())) {
        _printf("I'm father.\n");
    }
    else {
        _printf("I'm son.\n");
    }
    exit(0);
}
```

In [3]:
./task2.out

I'm father.
I'm son.


# Задание 3

Написать программу, в которой процесс порождает новый и позволяет порожденному процессу завершиться.

Оба процесса должны выводить свои идентификаторы:
- процесса,
- родительского процесса,
- группы процессов.

Проверить идентификатор и статус завершения порожденного процесса.

Выполнить данную программу и посмотреть ее результаты.

```c
// task3456.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>
#include <string.h>

#include "wraper.h"

int main(int argc, char ** argv) {
    pid_t ret_fork, ret_wait;
    int ret_code;
    int task;
    
    // size_t task_len;
    // char * ptr;

    // if (argc != 2) {
    //     _fprintf(stderr, "Usage: %s task\n",
    //              argv[0]);
    //     exit(1);
    // }

    // task = strtol(argv[1], &ptr, 0);
    // task_len = (size_t)(ptr - argv[1]);
    // if (strlen(argv[1]) != task_len ||
    //            task_len != 1 ||
    //            task < 3 ||
    //            task > 6) {
    //     _fprintf(stderr, "%s: incorrect task\n", argv[1]);
    //     exit(1);
    // }

    if (argc != 1) {
        _fprintf(stderr, "Usage: %s\n",
                 argv[0]);
        exit(1);
    }

    task = (int)(argv[0][6] - '0');
    _printf("task = %d\n\n", task);
    if (task < 3 || task > 6) {
        _fprintf(stderr, "%s: incorrect task\n", argv[0]);
        exit(1);
    }


    _printf ("I am father before fork.\n"
            "\tMy pid = %d\tppid = %d\tgid = %d\tpgrp = %d\n",
            getpid(), getppid(), getgid(), getpgrp());

    ret_fork = fork();
    
    if (ret_fork > 0) {
        if (task == 3)
            ret_wait= wait(&ret_code);
        _printf("I am father after fork.\n"
               "\tMy pid = %d\tppid = %d\tgid = %d\tpgrp = %d\n"
               "\tret_fork = %d\n",
               getpid(), getppid(), getgid(), getpgrp(), ret_fork);
        if (task == 3) {
            printf("\tret_wait = %d (shold be same as ret_fork)\n"
                   "\tret_code = %x\n"
                   "\tIFEXITED = %x\tEXITSTATUS=%x\n"
                   "\tIFSIGNALED = %x\tTERMSIG = %x\n",
                   ret_wait,
                   ret_code,
                   WIFEXITED(ret_code), WEXITSTATUS(ret_code),
                   WIFSIGNALED(ret_code), WTERMSIG(ret_code));
        }
        if (task == 6) {
            pause();
        }
        exit(0);
    }

    if (ret_fork == 0) {
        if (task == 4)
            sleep(1);
        _printf("I am son.\n"
               "\tMy pid = %d\tppid = %d\tgid = %d\tpgrp = %d\n"
               "\tret_fork = %d\n",
               getpid(), getppid(), getgid(), getpgrp(), ret_fork);
        if (task == 6) {
            pause();
        }
        exit(123);
    }
}
```

In [4]:
./task3.out

I am father before fork.
	My pid = 32392	ppid = 32326	gid = 1000	pgrp = 32392
I am son.
	My pid = 32393	ppid = 32392	gid = 1000	pgrp = 32392
	ret_fork = 0
I am father after fork.
	My pid = 32392	ppid = 32326	gid = 1000	pgrp = 32392
	ret_fork = 32393
	ret_wait = 32393 (shold be same as ret_fork)
	ret_code = 7b00
	IFEXITED = 1	EXITSTATUS=7b
	IFSIGNALED = 0	TERMSIG = 0


# Задание 4

Изменить программу Задания 3 так, чтобы родительский процесс завершился раньше порожденного.

Какой процесс становится родительским для порожденного после того, как исходный родительский процесс завершился?

In [5]:
./task4.out

I am father before fork.
	My pid = 32394	ppid = 32326	gid = 1000	pgrp = 32394
I am father after fork.
	My pid = 32394	ppid = 32326	gid = 1000	pgrp = 32394
	ret_fork = 32395


In [6]:
sleep 1

I am son.
	My pid = 32395	ppid = 9038	gid = 1000	pgrp = 32394
	ret_fork = 0


# Задание 5

Изменить программу Задания 3 так, чтобы родительский процесс выполнялся, не ожидая завершения порожденного процесса.

Что произойдет с порожденным процессом, когда он завершится?

Как убить зомби?

In [7]:
./task5.out

I am father before fork.
	My pid = 32401	ppid = 32326	gid = 1000	pgrp = 32401
I am father after fork.
	My pid = 32401	ppid = 32326	gid = 1000	pgrp = 32401
	ret_fork = 32402
I am son.
	My pid = 32402	ppid = 32401	gid = 1000	pgrp = 32401
	ret_fork = 0


# Задание 6

Изменить программу Задания 3 так, чтобы оба процесса выполняли системный вызов pause после вывода идентификаторов.

Запустить программу в фоновом режиме и посмотрите (с помощью ps), какие процессы существуют для данного пользователя.

Выполнить kill -9 pid для родительского процесса. Что произойдет с порожденным?

После очистки текущего сеанса, вновь выполнить программу, но уже в интерактивном режиме. Нажать на терминале клавишу CTRL-C. Что произойдет с родительским и порожденным процессами?

Можно ли получить такой же результат, если нажать клавишу прерывания после того, как родительский процесс завершится?

In [8]:
echo "do it in term."

do it in term.


# Задание 7

Модифицировать программу, включив в нее setpgrp в порожденный процесс до того, как он будет выполнять pause.

Повлияет ли на порожденный процесс нажатие клавиши прерывания в тот момент, когда родительский процесс "спит"?

Будет ли показан порожденный процесс в выводе команды ps?

In [9]:
echo "do it in term."

do it in term.


# Задание 8

Открыть файл (достаточно большого размера), доступный для чтения. Имя файла передается из командной строки.

После открытия файла создать параллельный процесс с помощью только fork. В обоих процессах создать свои файлы для записи, читать информацию из общего файла и копировать ее в собственные выходные файлы (не на экран).

Вывести на экран содержимое полученных файлов из порожденного процесса по окончании записи в файл и из родительского процесса, дождавшись завершения порожденного процесса.

Посмотреть, что изменится, если читаемую процессами информацию сразу выводить на экран.

```c
// task89.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <string.h>
#include <fcntl.h>

#include "wraper.h"

int main(int argc, char ** argv) {
    int task;
    int fin, fout;
    pid_t ret_fork;

    if (argc != 4 && argc != 2) {
        _fprintf(stderr,
                 "Usage: %s input output_father output_son\n"
                 "\t(output - optional)\n",
                 argv[0]);
        exit(1);
    }

    task = (int)(argv[0][6] - '0');
    if (task < 8 || task > 9) {
        _fprintf(stderr, "%s: incorrect task\n", argv[0]);
        exit(1);
    }

    if (task == 9)
        ret_fork = _fork();

    fin = _open(argv[1], O_RDONLY);

    if (task == 8)
        ret_fork = _fork();

    fout = argc == 4 ? _open(argv[2 + !ret_fork], O_WRONLY | O_CREAT | O_TRUNC , 0600)
         : FD_STD_OUT;
    rw_small_buf(fin, fout);

    exit(0);
}
```

In [10]:
./task8.out ~/.bashrc f s
cat f s
echo "/************************************************/"
./task8.out ~/.bashrc
sleep 1

# ~/.bashrc: executed by bash(1) for non-login shells.
# see /usr/share/doc/bash/examples/startup-files (in the package bash-doc)
# for examples

# If not running interactively, don't do anything
case $- in
    *i*) ;;
      *) return;;
esac

# don't put d lines or lines starting with space in the history.
# See bash(1re options
HISTCONTROL=ih

# apphe histo don't o it
shoptappend
etting hength seZE and HIZE in bISTSIZE=TFILESIZ# check ow size ch commaif necesupdate ts of LINOLUMNS.
 checkwi If set,tern "**n a pathansion cill
# mafiles anr more des and sories.
# globstae less mndly fort input ee lessp -x /usrspipe ] "$(SHELL lesspipet variatifying ot you wused in pt belowz "${debot:-}" ] /etc/deoot ]; tdebian_ccat /etcchroot)
t a fanc (non-coess we kwant" coe "$TERM xterm-c56color)rompt=ye

# uncor a colopt, if tnal has bility;  off by to not dthe usercus in al windowd be on ut of conot on tt
#forcerompt=ye-n "$for_prompt"
    if r/bin/tptput setdev/null# We havsupport;it's comith Ecma

# Задание 9

Выполнить Задания 8 при условии, что общий файл для чтения открывается в каждом из порожденных процессов.

In [11]:
./task9.out ~/.bashrc f s
diff f s
rm f s

# Задание 10

Создать (с помощью связки fork - exec) параллельный процесс. Имя исполняемого файла для exec передается с помощью аргумента командной строки.

Передать в порожденный процесс некоторую информацию через список параметров (список аргументов в функции main).

Каждый процесс (и родительский, и порожденный) должен вывести на экран список переданных ему аргументов и свое окружение.

```c
// task10.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

#include "wraper.h"

const char *const red = "\033[31m";
const char *const reset = "\033[0m";

int main(int argc, char ** argv, char ** envp) {

    _printf("%sArguments:%s\n", red, reset);
    for (int i = 0; i < argc; i++)
        _printf("\t%s\n", argv[i]);

    _printf("\n%sEnvironment:%s\n", red, reset);
    for (int i = 0; envp[i]; i++)
        _printf("\t%s\n", envp[i]);

    if (!_fork()) {
        execve(argv[1], argv + 1, NULL);
        perror(argv[1]);
    }

    exit(0);
}
```

In [12]:
./task10.out ./task10.out 67f66f6f6 6f f 6 6f6f 6ff6

Arguments:
	./task10.out
	./task10.out
	67f66f6f6
	6f
	f
	6
	6f6f
	6ff6

Environment:
	SHELL=/bin/bash
	SESSION_MANAGER=local/MSI-Desktop:@/tmp/.ICE-unix/9436,unix/MSI-Desktop:/tmp/.ICE-unix/9436
	QT_ACCESSIBILITY=1
	SNAP_REVISION=206
	XDG_CONFIG_DIRS=/etc/xdg/xdg-ubuntu:/etc/xdg
	PYTHONUNBUFFERED=1
	XDG_MENU_PREFIX=gnome-
	XDG_CONFIG_DIRS_VSCODE_SNAP_ORIG=/etc/xdg/xdg-ubuntu:/etc/xdg
	GNOME_DESKTOP_SESSION_ID=this-is-deprecated
	SNAP_REAL_HOME=/home/alexander
	SNAP_USER_COMMON=/home/alexander/snap/code/common
	FONTCONFIG_PATH=/etc/fonts
	GNOME_SHELL_SESSION_MODE=ubuntu
	SSH_AUTH_SOCK=/run/user/1000/keyring/ssh
	XDG_DATA_HOME=/home/alexander/snap/code/206/.local/share
	MEMORY_PRESSURE_WRITE=c29tZSAyMDAwMDAgMjAwMDAwMAA=
	PYTHON_FROZEN_MODULES=on
	ELECTRON_RUN_AS_NODE=1
	XMODIFIERS=@im=ibus
	DESKTOP_SESSION=ubuntu
	GDK_PIXBUF_MODULE_FILE=/home/alexander/snap/code/common/.cache/gdk-pixbuf-loaders.cache
	BAMF_DESKTOP_FILE_HINT=/var/lib/snapd/desktop/applications/code_code.desktop
	GTK_MODU

: 1

# Задание 11

Выполнить из программы на Си какую-либо команду Shell (cp или ls):
- с помощью вызовов fork-exec,
- с помощью библиотечной функции system.

Необходимые для команды данные передать через аргументы командной строки.

```c
// task11.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <string.h>

#include "wraper.h"

const char *const red = "\033[31m";
const char *const reset = "\033[0m";

int main(int argc, char ** argv) {
    
    char command[4096] = "";

    if (argc < 2) {
        _fprintf(stderr, "Usage: %s command args ...\n", argv[0]);
        exit(1);
    }

    if (!_fork()) {
        _printf("%sSon:%s\n", red, reset);
        execvp(argv[1], argv + 1);
        perror(argv[1]);
        exit(1);
    }

    for (int i = 1; i < argc; i++) {
        strcat(command, argv[i]);
        strcat(command, " ");
    }

    _wait(NULL);
    _printf("%sFather:%s\n", red, reset);
    system(command);

    exit(0);
}
```

In [ ]:
./task11.out ls -l -a -F --color=auto

Son:
total 260
drwxrwxr-x 2 alexander alexander  4096 Sep 30 21:07 ./
drwxrwxr-x 3 alexander alexander  4096 Sep 29 16:18 ../
-rwxrwxr-x 1 alexander alexander 17104 Sep 30 20:28 a.out*
-rw-rw-r-- 1 alexander alexander   562 Sep 30 20:32 task10.c
-rw-rw-r-- 1 alexander alexander   751 Sep 30 21:00 task11.c
-rwxrwxr-x 1 alexander alexander 17280 Sep 30 21:07 task11.out*
-rw-rw-r-- 1 alexander alexander   848 Sep 30 00:11 task1.c
-rwxrwxr-x 1 alexander alexander 17176 Sep 30 17:26 task1.out*
-rw-rw-r-- 1 alexander alexander   245 Sep 30 00:30 task2.c
-rwxrwxr-x 1 alexander alexander 16928 Sep 30 17:26 task2.out*
-rw-rw-r-- 1 alexander alexander  2342 Sep 30 01:00 task34567.c
-rwxrwxr-x 5 alexander alexander 17296 Sep 30 17:26 task3.out*
-rwxrwxr-x 5 alexander alexander 17296 Sep 30 17:26 task4.out*
-rwxrwxr-x 5 alexander alexander 17296 Sep 30 17:26 task5.out*
-rwxrwxr-x 5 alexander alexander 17296 Sep 30 17:26 task6.out*
-rwxrwxr-x 5 alexander alexander 17296 Sep 30 17:26 task7.out*
-rw-